In [12]:
import csv
import os
import pandas as pd
import numpy as np

In [13]:
url = r"C:\Users\owner\Desktop\Files_Deep_Learning\NeuraGuide\AI_Tools.csv"

def load_data(path):
    data = pd.read_csv(path)
    return data


df = load_data(url)
df = df.copy()

In [14]:
df.columns

Index(['Tool Name', 'Category', 'Primary Function', 'Description', 'Website',
       'Pricing Model', 'Key Features', 'Target Users', 'Launch Year',
       'Company', 'category_rank', 'ID', 'Category_code', 'average_rating',
       'review_count'],
      dtype='object')

In [15]:
df.head(1)

,Tool Name,Category,Primary Function,Description,Website,Pricing Model,Key Features,Target Users,Launch Year,Company,category_rank,ID,Category_code,average_rating,review_count
0,Kaedim,3D,3D,Transform 2D images into high-quality 3D model...,https://www.kaedim3d.com,Paid,See website,General,Unknown,Unknown,1,1,0,0.0,0


In [16]:
df.shape

(54910, 15)

## Identify and Remove Duplicate Tools

In [17]:
def check_duplicates(df):
    duplicates = df.duplicated().sum()
    if duplicates > 0:
        print(f"Duplicated rows: {duplicates}")
        # df = df.drop_duplicates(inplace=True)
        # dropped = df.duplicated().sum()
        # print(f"\nDuplicates: {dropped}")
    else:
        print("There are no duplicates")

check_duplicates(df)

There are no duplicates


In [20]:
# Detect duplicate entries using Tool Name, Company, and Website. Decide which record to keep and remove the rest using deterministic logic.

class DuplicateHandler:
    def __init__(self, df):
        self.df = df
        self.duplicate_keys = ['Tool Name', 'Company', 'Website']
        
    def find_duplicates(self):
        """Identify duplicate groups"""
        return self.df[self.df.duplicated(subset=self.duplicate_keys, keep=False)]
    
    def rank_records(self, group):
        """Score records: higher is better"""
        scores = pd.DataFrame(index=group.index)
        scores['completeness'] = group.notna().sum(axis=1)
        scores['reviews'] = group['review_count'].fillna(0)
        scores['rating'] = group['average_rating'].fillna(0)
        scores['recency'] = group['Launch Year'].fillna(0)
        scores['position'] = -np.arange(len(group))  # negative for ascending
        return scores.sum(axis=1).idxmax()
    
    def remove_duplicates(self):
        """Keep best record per duplicate group"""
        dupes = self.find_duplicates()
        if dupes.empty:
            print("There are no duplicates")
            return self.df, pd.DataFrame()
        
        keep_indices = dupes.groupby(self.duplicate_keys, dropna=False).apply(self.rank_records)
        removed = self.df[self.df.index.isin(dupes.index) & ~self.df.index.isin(keep_indices)]
        cleaned = self.df[~self.df.index.isin(dupes.index) | self.df.index.isin(keep_indices)]
        
        return cleaned.reset_index(drop=True), removed

# Usage:
handler = DuplicateHandler(df)
cleaned_df, removed_records = handler.remove_duplicates()

There are no duplicates


In [19]:
cleaned_df.shape

(54910, 15)